In [ ]:
import numpy as np
import pandas as pd


# get data


In [ ]:
host_tasks_df = pd.read_csv("https://osf.io/bd8h6/download", sep=" ", header=None).dropna(axis=1)
parasite_tasks_df = pd.read_csv("https://osf.io/8y5vd/download", sep=" ", header=None).dropna(axis=1)


In [ ]:
host_occupancy = host_tasks_df.to_numpy().flatten() != -1
host_occupancy


In [ ]:
parasite_occupancy = parasite_tasks_df.to_numpy().flatten() != -1
parasite_occupancy


In [ ]:
host_tasks = host_tasks_df.to_numpy().flatten()
host_tasks[~host_occupancy] = 0
host_tasks


In [ ]:
parasite_tasks = parasite_tasks_df.to_numpy().flatten()
parasite_tasks[~parasite_occupancy] = 0
parasite_tasks


In [ ]:
host_task_counts = np.array([
    int(val).bit_count()
    for val in host_tasks
])
host_task_counts


In [ ]:
parasite_task_counts = np.array([
    int(val).bit_count()
    for val in parasite_tasks
])
parasite_task_counts


In [ ]:
shared_tasks = host_tasks & parasite_tasks
shared_tasks


In [ ]:
shared_task_counts = np.array([
    int(val).bit_count()
    for val in shared_tasks
])
shared_task_counts


In [ ]:
host_task_tally = {}
for i in range(64):
    num_with_task = (host_tasks & (1 >> i)).sum()
    host_task_tally[i] = num_with_task

str(host_task_tally)


In [ ]:
parasite_task_tally = {}
for i in range(64):
    num_with_task = (parasite_tasks & (1 >> i)).sum()
    parasite_task_tally[i] = num_with_task

str(parasite_task_tally)


# get task counts


In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

array1 = host_task_counts[:20]
array2 = shared_task_counts[:20]
array3 = parasite_task_counts[:20]

n = len(array1)

# Create a bipartite graph
B = nx.Graph()
left_nodes = [f'l{i}' for i in range(n)]
right_nodes = [f'r{i}' for i in range(n)]

B.add_nodes_from(left_nodes, bipartite=0)
B.add_nodes_from(right_nodes, bipartite=1)

for i in range(n):
    B.add_edge(left_nodes[i], right_nodes[i], width=array2[i])

# Create a dictionary to store the positions
pos = dict()

# Calculate the positions for the top nodes
for i, node in enumerate(left_nodes):
    pos[node] = (i, 1)

# Calculate the positions for the bottom nodes
for i, node in enumerate(right_nodes):
    pos[node] = (i, 0)

node_sizes_left = dict(zip(left_nodes, array1))
node_sizes_right = dict(zip(right_nodes, array3))
node_sizes = {**node_sizes_left, **node_sizes_right}
edge_widths = nx.get_edge_attributes(B, 'width')

print("graph prepared")

plt.figure(figsize=(8, 6))
nx.draw_networkx_nodes(B, pos, node_size=[node_sizes[node] * 100 for node in B.nodes()])
nx.draw_networkx_edges(B, pos, width=[edge_widths[edge] for edge in B.edges()])
nx.draw_networkx_labels(B, pos, font_size=12)

plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
def get_set_bit_indices(n):
    indices = []
    index = 0
    while n:
        if n & 1:
            indices.append(index)
        n >>= 1
        index += 1
    return indices


In [ ]:
from collections import Counter
import itertools as it

from keyname import keyname as kn
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

n = 9  # num tasks

# Create a bipartite graph
B = nx.Graph()
left_nodes = [f'host_task={i}' for i in range(n)]
right_nodes = [f'parasite_task={i}' for i in range(n)]

B.add_nodes_from(left_nodes, bipartite=0)
B.add_nodes_from(right_nodes, bipartite=1)

shared_task_pairs = Counter()
for shared_task in shared_tasks:
    set_indices = get_set_bit_indices(shared_task)
    for index in set_indices:
        shared_task_pairs[(index, index)] += 1
    for pair in it.combinations(
        set_indices, r=2
    ):
        shared_task_pairs[frozenset(pair)] += 1

for (p1, p2), count in shared_task_pairs.items():
    B.add_edge(
        f"host_task={p1}",
        f"parasite_task={p2}",
        width=count
    )

# Create a dictionary to store the positions
pos = dict()

# Calculate the positions for the top nodes
for i, node in enumerate(left_nodes):
    pos[node] = (i, 1)

# Calculate the positions for the bottom nodes
for i, node in enumerate(right_nodes):
    pos[node] = (i, 0)

node_sizes_left = {
    node : host_task_tally[
        int(kn.unpack(node)["host_task"])
    ]
    for node in left_nodes
}
node_sizes_right = {
    node : parasite_task_tally[
        int(kn.unpack(node)["parasite_task"])
    ]
    for node in right_nodes
}
node_sizes = {**node_sizes_left, **node_sizes_right}
edge_widths = nx.get_edge_attributes(B, 'width')

print("graph prepared")

plt.figure(figsize=(8, 6))
nx.draw_networkx_nodes(B, pos, node_size=[node_sizes[node] / 3 for node in B.nodes()])
nx.draw_networkx_edges(B, pos, width=[edge_widths[edge] / 500 for edge in B.edges()])
nx.draw_networkx_labels(
    B,
    pos,
    {n : ''.join(char for char in n if char.isdigit()) for n in B},
    font_size=12,
)

# Add a top label and a bottom label
top_label = "Host Tasks"
bottom_label = "Parasite Tasks"

# Calculate the x-position for the labels (centered)
label_x = (n - 1) / 2

plt.text(label_x, 1.15, top_label, ha='center', fontsize=14, fontweight='bold')
plt.text(label_x, -0.15, bottom_label, ha='center', fontsize=14, fontweight='bold')

plt.axis('off')
plt.tight_layout()
plt.show()
